In [46]:
import pandas as pd
df = pd.read_csv("result/Spark_2k.log_structured.csv")
print(df.head())

   LineId      Date      Time Level                              Component  \
0       1  17/06/20  05:10:40  INFO  executor.CoarseGrainedExecutorBackend   
1       2  17/06/20  05:10:40  INFO                  spark.SecurityManager   
2       3  17/06/20  05:10:40  INFO                        com.org.sravani   
3       4  17/06/20  05:10:40  INFO                  spark.SecurityManager   
4       5  17/06/20  05:10:40  INFO                  spark.SecurityManager   

                                             Content   EventId  \
0    Registered signal handlers for [TERM, HUP, INT]  932c104e   
1                   Changing view acls to: yarn,curi  7bb9d001   
2                                  Finishing the job  93322a10   
3                 Changing modify acls to: yarn,curi  7bb9d001   
4   SecurityManager: authentication disabled; ui ...  29bc3dfc   

                                       EventTemplate ParameterList  
0    Registered signal handlers for [TERM, HUP, INT]            [

In [43]:
from elasticsearch.helpers import parallel_bulk
from collections import deque
from typing import Any, Dict, Generator, List, Tuple, Union
from elasticsearch import Elasticsearch
import uuid

DEFAULT_CHUNK_SIZE = 10000
DEFAULT_THREAD_COUNT = 4

def action_generator(
    pd_df: pd.DataFrame,
    es_dest_index: str,
) -> Generator[Dict[str, Any], None, None]:
    for row in pd_df.iterrows():
        values = row[1].to_dict()
        id = row[0]
        #action = {"_index": es_dest_index, "_source": values, "_id": str(id)}
        action = {
            "_op_type": "update",
            "_index": es_dest_index,
            "_id": str(uuid.uuid4()),
            "_source": {
                "doc": values,
                "doc_as_upsert": True
            }
        }
        yield action

def pushData(es_client:Union[str, List[str], Tuple[str, ...], Elasticsearch],pd_df: pd.DataFrame,es_dest_index:str) :
    try:
        deque(
        parallel_bulk(
            client=es_client,
            actions=action_generator(
                pd_df, es_dest_index
            ),
            chunk_size=int(DEFAULT_CHUNK_SIZE / DEFAULT_THREAD_COUNT),
        ),
        maxlen=0,
    )
    except Exception as e:
        print(e)

In [44]:
es_client=Elasticsearch(hosts='http://localhost:9200', http_auth=('elastic', 'mangena'))
es_index = "logs-pyspark"

In [47]:
pushData(es_client,df,es_index)

In [42]:
for row in df.iterrows(): 
        print(str(row[1]["LineId"])+row[1]["Date"]+row[1]["Time"])
        break

117/06/2305:10:40
